In [35]:
import re
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch

from tokenizers import decoders

In [25]:
model = AutoModelForTokenClassification.from_pretrained("bert_ner_finetuned_iliad-with-gpu.model")

In [27]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [28]:
label_list = ['O', 'B-CLEntity', 'I-CLEntity', 'L-CLEntity', 'U-CLEntity']

In [29]:
sequence = """The man for wisdom’s various arts renown’d,
Long exercised in woes, O Muse! resound;
Who, when his arms had wrought the destined fall
Of sacred Troy, and razed her heaven-built wall,
Wandering from clime to clime, observant stray’d,
Their manners noted, and their states survey’d,
On stormy seas unnumber’d toils he bore,
Safe with his friends to gain his natal shore:
Vain toils! their impious folly dared to prey
On herds devoted to the god of day;
The god vindictive doom’d them never more
(Ah, men unbless’d!) to touch that natal shore.
Oh, snatch some portion of these acts from fate,
Celestial Muse! and to our world relate.
"""

In [14]:
odyssey_lines = []
book = open("../example-texts/odyssey.txt")
for line in book:
    line = line.strip()
    odyssey_lines.append(line)

In [18]:
odyssey_lines = [line for line in odyssey_lines if line]

In [19]:
odyssey_lines[:10]

['The man for wisdom’s various arts renown’d,',
 'Long exercised in woes, O Muse! resound;',
 'Who, when his arms had wrought the destined fall',
 'Of sacred Troy, and razed her heaven-built wall,',
 'Wandering from clime to clime, observant stray’d,',
 'Their manners noted, and their states survey’d,',
 'On stormy seas unnumber’d toils he bore,',
 'Safe with his friends to gain his natal shore:',
 'Vain toils! their impious folly dared to prey',
 'On herds devoted to the god of day;']

In [30]:
tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sequence)))
inputs = tokenizer.encode(sequence, return_tensors="pt")

outputs = model(inputs).logits
predictions = torch.argmax(outputs, dim=2)

In [32]:
## https://huggingface.co/docs/tokenizers/python/latest/pipeline.html
# tokenizer.decoder = decoders.WordPiece()
# tokenizer.decode(outputs.ids)

AttributeError: can't set attribute

In [8]:
predictions

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 4, 0,
         0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 1, 2, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0]])

In [9]:
for token, prediction in zip(tokens, predictions[0].numpy()):
     print((token, model.config.id2label[prediction]))

('[CLS]', 'LABEL_0')
('The', 'LABEL_0')
('man', 'LABEL_0')
('for', 'LABEL_0')
('wisdom', 'LABEL_0')
('’', 'LABEL_0')
('s', 'LABEL_0')
('various', 'LABEL_0')
('arts', 'LABEL_0')
('re', 'LABEL_0')
('##no', 'LABEL_0')
('##wn', 'LABEL_0')
('’', 'LABEL_0')
('d', 'LABEL_0')
(',', 'LABEL_0')
('Long', 'LABEL_4')
('exercised', 'LABEL_0')
('in', 'LABEL_0')
('w', 'LABEL_0')
('##oes', 'LABEL_0')
(',', 'LABEL_0')
('O', 'LABEL_0')
('Muse', 'LABEL_4')
('!', 'LABEL_0')
('re', 'LABEL_0')
('##sound', 'LABEL_0')
(';', 'LABEL_0')
('Who', 'LABEL_4')
(',', 'LABEL_0')
('when', 'LABEL_0')
('his', 'LABEL_0')
('arms', 'LABEL_0')
('had', 'LABEL_0')
('wrought', 'LABEL_0')
('the', 'LABEL_0')
('destined', 'LABEL_0')
('fall', 'LABEL_0')
('Of', 'LABEL_0')
('sacred', 'LABEL_0')
('Troy', 'LABEL_4')
(',', 'LABEL_0')
('and', 'LABEL_0')
('r', 'LABEL_0')
('##azed', 'LABEL_0')
('her', 'LABEL_0')
('heaven', 'LABEL_0')
('-', 'LABEL_0')
('built', 'LABEL_0')
('wall', 'LABEL_0')
(',', 'LABEL_0')
('Wan', 'LABEL_1')
('##dering', '

In [21]:
for line in odyssey_lines:
    tokens_line = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(line)))
    inputs_line = tokenizer.encode(line, return_tensors="pt")

    outputs_line = model(inputs_line).logits
    predictions_line = torch.argmax(outputs_line, dim=2)
    for token, prediction in zip(tokens_line, predictions_line[0].numpy()):
         print((token, model.config.id2label[prediction]))

('[CLS]', 'LABEL_0')
('The', 'LABEL_0')
('man', 'LABEL_0')
('for', 'LABEL_0')
('wisdom', 'LABEL_0')
('’', 'LABEL_0')
('s', 'LABEL_0')
('various', 'LABEL_0')
('arts', 'LABEL_0')
('re', 'LABEL_0')
('##no', 'LABEL_0')
('##wn', 'LABEL_0')
('’', 'LABEL_0')
('d', 'LABEL_0')
(',', 'LABEL_0')
('[SEP]', 'LABEL_0')
('[CLS]', 'LABEL_0')
('Long', 'LABEL_0')
('exercised', 'LABEL_0')
('in', 'LABEL_0')
('w', 'LABEL_0')
('##oes', 'LABEL_0')
(',', 'LABEL_0')
('O', 'LABEL_0')
('Muse', 'LABEL_4')
('!', 'LABEL_0')
('re', 'LABEL_0')
('##sound', 'LABEL_0')
(';', 'LABEL_0')
('[SEP]', 'LABEL_0')
('[CLS]', 'LABEL_0')
('Who', 'LABEL_0')
(',', 'LABEL_0')
('when', 'LABEL_0')
('his', 'LABEL_0')
('arms', 'LABEL_0')
('had', 'LABEL_0')
('wrought', 'LABEL_0')
('the', 'LABEL_0')
('destined', 'LABEL_0')
('fall', 'LABEL_0')
('[SEP]', 'LABEL_0')
('[CLS]', 'LABEL_0')
('Of', 'LABEL_0')
('sacred', 'LABEL_0')
('Troy', 'LABEL_4')
(',', 'LABEL_0')
('and', 'LABEL_0')
('r', 'LABEL_0')
('##azed', 'LABEL_0')
('her', 'LABEL_0')
('he

('[CLS]', 'LABEL_0')
('The', 'LABEL_0')
('rising', 'LABEL_0')
('and', 'LABEL_0')
('descending', 'LABEL_0')
('sun', 'LABEL_0')
('surveys', 'LABEL_0')
(')', 'LABEL_0')
(';', 'LABEL_0')
('[SEP]', 'LABEL_0')


KeyboardInterrupt: 

In [39]:
pattern = r' ([A-Z].[a-z]+)'
re.compile(pattern)

def get_annotations(text, pattern):
    annotations = []
    for match in re.finditer(pattern, text):
        label_dic = dict()
        label_dic['start'] = match.start()
        label_dic['end'] = match.end()
        label_dic['label'] = 'CLEntity' # Entity starting with a capital letter
        annotations.append(label_dic)
    return annotations

In [40]:
json_data = []
book = open("../example-texts/odyssey.txt")
for line in book:
    line = line.strip()
    
    line_data = dict()
    line_data['content'] = line
    line_data['annotations'] = get_annotations(line, pattern)
    json_data.append(line_data)

In [42]:
# For testing the result. Need a way to test whether tuned bert is accurately identifying CLEntities
json_data[:10] 

[{'content': '', 'annotations': []},
 {'content': '', 'annotations': []},
 {'content': 'The man for wisdom’s various arts renown’d,', 'annotations': []},
 {'content': 'Long exercised in woes, O Muse! resound;',
  'annotations': [{'start': 25, 'end': 30, 'label': 'CLEntity'}]},
 {'content': 'Who, when his arms had wrought the destined fall',
  'annotations': []},
 {'content': 'Of sacred Troy, and razed her heaven-built wall,',
  'annotations': [{'start': 9, 'end': 14, 'label': 'CLEntity'}]},
 {'content': 'Wandering from clime to clime, observant stray’d,',
  'annotations': []},
 {'content': 'Their manners noted, and their states survey’d,',
  'annotations': []},
 {'content': 'On stormy seas unnumber’d toils he bore,', 'annotations': []},
 {'content': 'Safe with his friends to gain his natal shore:',
  'annotations': []}]